## Setup

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import gc

from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm
from fastkde.fastKDE import fastKDE
from scipy.stats import linregress
from scipy.signal import butter, filtfilt
from scipy.signal import correlate
from scipy.stats import pearsonr
from scipy.signal import find_peaks

from harp_resources import process, utils
from sleap import load_and_process as lp

# symbols to use ✅ ℹ️ ⚠️ ❗

In [ ]:
# set up variables and load data 
############################################################################################################

plot_timeseries = False
score_cutoff = 0.2 # for filtering out inferred points with low confidence, they get interpolated 
outlier_sd_threshold = 10 # for removing outliers from the data, they get interpolated 
NaNs_removed = False # for checking if NaNs already removed in the notebook
cutoff = 10  # Hz for pupil diameter filtering 

# for saccades
refractory_period = 0.1  # sec
k = 4  # for adaptive saccade threshold - Number of standard deviations (adjustable: 2-4 range works well) 
onset_offset_fraction = 0.2  # to determine saccade onset, works on the velocity peak;
plot_saccade_detection_QC = True

data_path = Path('/Users/rancze/Documents/Data/vestVR/20250409_Cohort3_rotation/Visual_mismatch_day4/B6J2782-2025-04-28T14-22-03') 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/No_iso_correction/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03') # only has sleap data 1
save_path = data_path.parent / f"{data_path.name}_processedData"

print ("\n❗ 20251025 NOT sure I understand this Ede ---- if SleapData.csv was already saved in the VideoData folder, this may break. Delete the file if you want to rerun processing\n")
VideoData1, VideoData2, VideoData1_Has_Sleap, VideoData2_Has_Sleap = lp.load_videography_data(data_path)

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

if VideoData1_Has_Sleap:
    VideoData1 = VideoData1.drop(columns=['track']) # drop the track column as it is empty
    coordinates_dict1_raw=lp.get_coordinates_dict(VideoData1, columns_of_interest)
    FPS_1 = 1 / VideoData1["Seconds"].diff().mean()  # frame rate for VideoData1 TODO where to save it, is it useful?
    print(f"FPS_1: {FPS_1}")

if VideoData2_Has_Sleap:
    VideoData2 = VideoData2.drop(columns=['track']) # drop the track column as it is empty
    coordinates_dict2_raw=lp.get_coordinates_dict(VideoData2, columns_of_interest)
    FPS_2 = 1 / VideoData2["Seconds"].diff().mean()  # frame rate for VideoData2
    print(f"FPS_2: {FPS_2}")


In [ ]:
# plot timeseries of coordinates in browser for both VideoData1 and VideoData2
############################################################################################################
if plot_timeseries:
    print(f'⚠️ Check for long discontinuities and outliers in the data, we will try to deal with them later')
    print(f'ℹ️ Figures open in browser window, takes a bit of time.')

    # Helper list variables
    subplot_titles = (
        "X coordinates for pupil centre and left-right eye corner",
        "Y coordinates for pupil centre and left-right eye corner",
        "X coordinates for iris points",
        "Y coordinates for iris points"
    )
    eye_x = ['left.x', 'center.x', 'right.x']
    eye_y = ['left.y', 'center.y', 'right.y']
    iris_x = ['p1.x', 'p2.x', 'p3.x', 'p4.x', 'p5.x', 'p6.x', 'p7.x', 'p8.x']
    iris_y = ['p1.y', 'p2.y', 'p3.y', 'p4.y', 'p5.y', 'p6.y', 'p7.y', 'p8.y']

    # --- VideoData1 ---
    if VideoData1_Has_Sleap:
        fig1 = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=subplot_titles
        )

        # Row 1: left.x, center.x, right.x
        for col in eye_x:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=1, col=1)
        # Row 2: left.y, center.y, right.y
        for col in eye_y:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=2, col=1)
        # Row 3: p1.x ... p8.x
        for col in iris_x:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=3, col=1)
        # Row 4: p1.y ... p8.y
        for col in iris_y:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=4, col=1)

        fig1.update_layout(
            height=1200,
            title_text="Time series subplots for coordinates [VideoData1]",
            showlegend=True
        )
        fig1.update_xaxes(title_text="Seconds", row=4, col=1)
        fig1.update_yaxes(title_text="X Position", row=1, col=1)
        fig1.update_yaxes(title_text="Y Position", row=2, col=1)
        fig1.update_yaxes(title_text="X Position", row=3, col=1)
        fig1.update_yaxes(title_text="Y Position", row=4, col=1)

        fig1.show(renderer='browser')

    # --- VideoData2 ---
    if VideoData2_Has_Sleap:
        fig2 = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=subplot_titles
        )
        # Row 1: left.x, center.x, right.x
        for col in eye_x:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=1, col=1)
        # Row 2: left.y, center.y, right.y
        for col in eye_y:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=2, col=1)
        # Row 3: p1.x ... p8.x
        for col in iris_x:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=3, col=1)
        # Row 4: p1.y ... p8.y
        for col in iris_y:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=4, col=1)

        fig2.update_layout(
            height=1200,
            title_text="Time series subplots for coordinates [VideoData2]",
            showlegend=True
        )
        fig2.update_xaxes(title_text="Seconds", row=4, col=1)
        fig2.update_yaxes(title_text="X Position", row=1, col=1)
        fig2.update_yaxes(title_text="Y Position", row=2, col=1)
        fig2.update_yaxes(title_text="X Position", row=3, col=1)
        fig2.update_yaxes(title_text="Y Position", row=4, col=1)

        fig2.show(renderer='browser')

In [ ]:
# QC plot XY coordinate distributions to visualize outliers 
############################################################################################################

columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

# Filter out NaN values and calculate the min and max values for X and Y coordinates for both dict1 and dict2

def min_max_dict(coordinates_dict):
    x_min = min([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].min() for col in columns_of_interest])
    x_max = max([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].max() for col in columns_of_interest])
    y_min = min([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].min() for col in columns_of_interest])
    y_max = max([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].max() for col in columns_of_interest])
    return x_min, x_max, y_min, y_max

# Only plot panels for 1 and 2 if VideoData1_Has_Sleap and/or VideoData2_Has_Sleap are true

# Compute min/max as before for global axes limits
if VideoData1_Has_Sleap:
    x_min1, x_max1, y_min1, y_max1 = min_max_dict(coordinates_dict1_raw)
if VideoData2_Has_Sleap:
    x_min2, x_max2, y_min2, y_max2 = min_max_dict(coordinates_dict2_raw)

# Use global min and max for consistency only if both VideoData1_Has_Sleap and VideoData2_Has_Sleap are True
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    x_min = min(x_min1, x_min2)
    x_max = max(x_max1, x_max2)
    y_min = min(y_min1, y_min2)
    y_max = max(y_max1, y_max2)
elif VideoData1_Has_Sleap:
    x_min, x_max, y_min, y_max = x_min1, x_max1, y_min1, y_max1
elif VideoData2_Has_Sleap:
    x_min, x_max, y_min, y_max = x_min2, x_max2, y_min2, y_max2
else:
    raise ValueError("Neither VideoData1 nor VideoData2 has Sleap data available.")

# Create the figure and axes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18, 12))
fig.suptitle('XY coordinate distribution of different points for VideoData1 (_dict1) and VideoData2 (_dict2) before outlier removal and NaN interpolation', fontsize=14)

# Define colormap for p1-p8
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange']

# Panel 1: left, right, center (dict1)
if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
    ax[0, 0].set_title('VideoData1 (_dict1): left, right, center')
    ax[0, 0].scatter(coordinates_dict1_raw['left.x'], coordinates_dict1_raw['left.y'], color='black', label='left', s=10)
    ax[0, 0].scatter(coordinates_dict1_raw['right.x'], coordinates_dict1_raw['right.y'], color='grey', label='right', s=10)
    ax[0, 0].scatter(coordinates_dict1_raw['center.x'], coordinates_dict1_raw['center.y'], color='red', label='center', s=10)
    ax[0, 0].set_xlim([x_min, x_max])
    ax[0, 0].set_ylim([y_min, y_max])
    ax[0, 0].set_xlabel('x coordinates (pixels)')
    ax[0, 0].set_ylabel('y coordinates (pixels)')
    ax[0, 0].legend(loc='upper right')
else:
    ax[0, 0].axis('off')

# Panel 2: p1 to p8 (dict1)
if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
    ax[0, 1].set_title('VideoData1 (_dict1): p1 to p8')
    for idx, col in enumerate(columns_of_interest[3:]):
        ax[0, 1].scatter(coordinates_dict1_raw[f'{col}.x'], coordinates_dict1_raw[f'{col}.y'], color=colors[idx], label=col, s=5)
    ax[0, 1].set_xlim([x_min, x_max])
    ax[0, 1].set_ylim([y_min, y_max])
    ax[0, 1].set_xlabel('x coordinates (pixels)')
    ax[0, 1].set_ylabel('y coordinates (pixels)')
    ax[0, 1].legend(loc='upper right')
else:
    ax[0, 1].axis('off')

# Panel 3: left, right, center (dict2)
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    ax[1, 0].set_title('VideoData2 (_dict2): left, right, center')
    ax[1, 0].scatter(coordinates_dict2_raw['left.x'], coordinates_dict2_raw['left.y'], color='black', label='left', s=10)
    ax[1, 0].scatter(coordinates_dict2_raw['right.x'], coordinates_dict2_raw['right.y'], color='grey', label='right', s=10)
    ax[1, 0].scatter(coordinates_dict2_raw['center.x'], coordinates_dict2_raw['center.y'], color='red', label='center', s=10)
    ax[1, 0].set_xlim([x_min, x_max])
    ax[1, 0].set_ylim([y_min, y_max])
    ax[1, 0].set_xlabel('x coordinates (pixels)')
    ax[1, 0].set_ylabel('y coordinates (pixels)')
    ax[1, 0].legend(loc='upper right')
else:
    ax[1, 0].axis('off')

# Panel 4: p1 to p8 (dict2)
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    ax[1, 1].set_title('VideoData2 (_dict2): p1 to p8')
    for idx, col in enumerate(columns_of_interest[3:]):
        ax[1, 1].scatter(coordinates_dict2_raw[f'{col}.x'], coordinates_dict2_raw[f'{col}.y'], color=colors[idx], label=col, s=5)
    ax[1, 1].set_xlim([x_min, x_max])
    ax[1, 1].set_ylim([y_min, y_max])
    ax[1, 1].set_xlabel('x coordinates (pixels)')
    ax[1, 1].set_ylabel('y coordinates (pixels)')
    ax[1, 1].legend(loc='upper right')
else:
    ax[1, 1].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()


In [ ]:
# QC for consecutive NaN and low confidence inference frames TODO - not sure what to do, what threshold to use to send it back to SLEAP inference 
############################################################################################################
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

# VideoData1 NaN analysis
if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
    all_nan_df = VideoData1[VideoData1[columns_of_interest].isnull().all(1)]
    all_nan_index_array = all_nan_df.index.values

    # print the groups of sequential NaNs
    group_counts = {'1-5': 0, '6-10': 0, '>10': 0}
    i = 1
    for group in lp.find_sequential_groups(all_nan_index_array):
        #print(f'NaN frame group {i} with {len(group)} elements')
        if 1 <= len(group) <= 5:
            group_counts['1-5'] += 1
        elif 6 <= len(group) <= 10:
            group_counts['6-10'] += 1
        else:
            group_counts['>10'] += 1
            print(f'\n⚠️ VideoData1 Framegroup {i} has {len(group)} consecutive all NaN frames  with indices {group}. If this is a long group, consider rerunning SLEAP inference.')
        i += 1

    print(f"\nVideoData1 - Framegroups with 1-5 consecutive all NaN frames: {group_counts['1-5']}")
    print(f"VideoData1 - Framegroups with 6-10 consecutive all NaN frames: {group_counts['6-10']}")
    print(f"VideoData1 - Framegroups with >10 consecutive all NaN frames: {group_counts['>10']}")

# VideoData2 NaN analysis
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    all_nan_df2 = VideoData2[VideoData2[columns_of_interest].isnull().all(1)]
    all_nan_index_array2 = all_nan_df2.index.values

    # print the groups of sequential NaNs for VideoData2
    group_counts2 = {'1-5': 0, '6-10': 0, '>10': 0}
    i = 1
    for group in lp.find_sequential_groups(all_nan_index_array2):
        #print(f'NaN frame group {i} with {len(group)} elements')
        if 1 <= len(group) <= 5:
            group_counts2['1-5'] += 1
        elif 6 <= len(group) <= 10:
            group_counts2['6-10'] += 1
        else:
            group_counts2['>10'] += 1
            print(f'\n⚠️ VideoData2 Framegroup {i} has {len(group)} consecutive all NaN frames  with indices {group}. If this is a long group, consider rerunning SLEAP inference.')
        i += 1

    print(f"\nVideoData2 - Framegroups with 1-5 consecutive all NaN frames: {group_counts2['1-5']}")
    print(f"VideoData2 - Framegroups with 6-10 consecutive all NaN frames: {group_counts2['6-10']}")
    print(f"VideoData2 - Framegroups with >10 consecutive all NaN frames: {group_counts2['>10']}")

############################################################################################################
# check if we can use some filtering on scores to remove bad frames
############################################################################################################

columns_of_interest = ['left.score','center.score','right.score','p1.score','p2.score','p3.score','p4.score','p5.score','p6.score','p7.score','p8.score']

# VideoData1 confidence score analysis
if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
    total_points1 = len(VideoData1)
    print(f'\nℹ️ VideoData1 - Top 3 columns with most frames below {score_cutoff} confidence score:')

    # Calculate statistics for all columns
    video1_stats = []
    for col in columns_of_interest:
        count_below_threshold = (VideoData1[col] < score_cutoff).sum()
        percentage_below_threshold = (count_below_threshold / total_points1) * 100

        # Find the longest consecutive series below threshold
        below_threshold = VideoData1[col] < score_cutoff
        longest_series = 0
        current_series = 0

        for value in below_threshold:
            if value:
                current_series += 1
                if current_series > longest_series:
                    longest_series = current_series
            else:
                current_series = 0

        video1_stats.append((col, count_below_threshold, percentage_below_threshold, longest_series))

    # Sort by count_below_threshold and show top 3
    video1_stats.sort(key=lambda x: x[1], reverse=True)
    for i, (col, count, percentage, longest) in enumerate(video1_stats[:3]):
        print(f"VideoData1 - #{i+1}: {col} | Values below {score_cutoff}: {count} ({percentage:.2f}%) | Longest consecutive frame series: {longest}")

# VideoData2 confidence score analysis
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    total_points2 = len(VideoData2)
    print(f'\nℹ️ VideoData2 - Top 3 columns with most frames below {score_cutoff} confidence score:')

    # Calculate statistics for all columns
    video2_stats = []
    for col in columns_of_interest:
        count_below_threshold = (VideoData2[col] < score_cutoff).sum()
        percentage_below_threshold = (count_below_threshold / total_points2) * 100
        
        # Find the longest consecutive series below threshold
        below_threshold = VideoData2[col] < score_cutoff
        longest_series = 0
        current_series = 0
        
        for value in below_threshold:
            if value:
                current_series += 1
                if current_series > longest_series:
                    longest_series = current_series
            else:
                current_series = 0
        
        video2_stats.append((col, count_below_threshold, percentage_below_threshold, longest_series))

    # Sort by count_below_threshold and show top 3
    video2_stats.sort(key=lambda x: x[1], reverse=True)
    for i, (col, count, percentage, longest) in enumerate(video2_stats[:3]):
        print(f"VideoData2 - #{i+1}: {col} | Values below {score_cutoff}: {count} ({percentage:.2f}%) | Longest consecutive frame series: {longest}")

In [ ]:
# Plotting the distribution of instance scores IN VIEW OF removing those as well! 
# Likely problem is that when instance score is low, that's because of a blink or similar, as there are long sequences of low scores
# which is perfeect to exclude and linearly interpolate (or keep it NaN?) -> but need to make sure it is really a blink an not model-issues 
# Now using an adaptive percentile threshold to ensure the maximum number of consecutively excluded frames <= max_consecutive_lowscore (default=6)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

max_consecutive_lowscore = 120  # in frames TODO make it depend on FPS_1 and FPS_2 

def find_longest_lowscore_sections(scores, threshold, top_n=5):
    """
    Finds the top_n longest consecutive segments where scores < threshold.
    Returns list of dict: {'start_idx', 'end_idx', 'length', 'start_time', 'end_time' (if available)}
    """
    is_low = scores < threshold
    diff = is_low.astype(int).diff().fillna(0)
    starts = np.where((diff == 1))[0]
    ends = np.where((diff == -1))[0]

    if is_low.iloc[0]:
        starts = np.insert(starts, 0, 0)
    if is_low.iloc[-1]:
        ends = np.append(ends, len(is_low))

    sections = []
    for s, e in zip(starts, ends):
        sections.append({'start_idx': s, 'end_idx': e-1, 'length': e-s})

    top_sections = sorted(sections, key=lambda x: x['length'], reverse=True)[:top_n]
    return top_sections

def find_percentile_for_consecutive_limit(scores, max_consecutive):
    """
    Binary search for the lowest percentile so that the maximum number of
    consecutively excluded frames (scores below the percentile) does not exceed max_consecutive.
    Returns: percentile and threshold value.
    """
    sorted_scores = scores.dropna().sort_values()
    n = len(sorted_scores)
    low, high = 0.0, 0.2  # Search up to the 20th percentile; adjust as needed
    tol = 0.0001

    found_pct = None
    found_value = None

    while (high - low) > tol:
        mid = (low + high) / 2
        thr = sorted_scores.quantile(mid)
        sections = find_longest_lowscore_sections(scores, thr, top_n=1)
        longest = sections[0]['length'] if sections else 0
        if longest > max_consecutive:
            high = mid
        else:
            found_pct = mid
            found_value = thr
            low = mid

    return found_pct, found_value

# ---- User parameter: maximum allowed consecutive low-score frames ----

# Only analyze for dataset(s) that exist
has_v1 = "VideoData1_Has_Sleap" in globals() and VideoData1_Has_Sleap
has_v2 = "VideoData2_Has_Sleap" in globals() and VideoData2_Has_Sleap

fig = None
if has_v1 or has_v2:
    plt.figure(figsize=(12,5))
    plot_index = 1

# Find adaptive percentile and threshold for each present VideoData
if has_v1:
    adaptive_pct_v1, adaptive_thr_v1 = find_percentile_for_consecutive_limit(VideoData1['instance.score'], max_consecutive_lowscore)
    plt.subplot(1, 2 if has_v2 else 1, plot_index)
    plt.hist(VideoData1['instance.score'].dropna(), bins=30, color='skyblue', edgecolor='black')
    plt.axvline(adaptive_thr_v1, color='red', linestyle='--', label=f'Adaptive threshold ({adaptive_pct_v1*100:.2f} percentile)\nlimit {max_consecutive_lowscore} consecutive')
    plt.title("Distribution of instance.score (VideoData1)")
    plt.xlabel("instance.score")
    plt.ylabel("Frequency")
    plt.legend()
    plot_index += 1

if has_v2:
    adaptive_pct_v2, adaptive_thr_v2 = find_percentile_for_consecutive_limit(VideoData2['instance.score'], max_consecutive_lowscore)
    plt.subplot(1, 2 if has_v1 else 1, plot_index)
    plt.hist(VideoData2['instance.score'].dropna(), bins=30, color='salmon', edgecolor='black')
    plt.axvline(adaptive_thr_v2, color='red', linestyle='--', label=f'Adaptive threshold ({adaptive_pct_v2*100:.2f} percentile)\nlimit {max_consecutive_lowscore} consecutive')
    plt.title("Distribution of instance.score (VideoData2)")
    plt.xlabel("instance.score")
    plt.ylabel("Frequency")
    plt.legend()
    plot_index += 1

if has_v1 or has_v2:
    plt.tight_layout()
    plt.show()

# Report the statistics for available VideoData
if has_v1:
    v1_num_low = (VideoData1['instance.score'] < adaptive_thr_v1).sum()
    print(f"\nVideoData1: Total number of points with instance.score < adaptive threshold: {v1_num_low}")
if has_v2:
    v2_num_low = (VideoData2['instance.score'] < adaptive_thr_v2).sum()
    print(f"VideoData2: Total number of points with instance.score < adaptive threshold: {v2_num_low}")

# Report the top 5 longest consecutive sections with instance.score < adaptive threshold
if has_v1:
    print(f"\nVideoData1: Top 5 longest consecutive sections where instance.score < adaptive threshold:")
    low_sections_v1 = find_longest_lowscore_sections(VideoData1['instance.score'], adaptive_thr_v1, top_n=5)
    for i, sec in enumerate(low_sections_v1, 1):
        start_idx = sec['start_idx']
        end_idx = sec['end_idx']
        start_time = VideoData1.index[start_idx] if isinstance(VideoData1.index, pd.DatetimeIndex) else start_idx
        end_time = VideoData1.index[end_idx] if isinstance(VideoData1.index, pd.DatetimeIndex) else end_idx
        print(f"  Section {i}: index {start_idx}-{end_idx} (length {sec['length']})")
if has_v2:
    print(f"\nVideoData2: Top 5 longest consecutive sections where instance.score < adaptive threshold:")
    low_sections_v2 = find_longest_lowscore_sections(VideoData2['instance.score'], adaptive_thr_v2, top_n=5)
    for i, sec in enumerate(low_sections_v2, 1):
        start_idx = sec['start_idx']
        end_idx = sec['end_idx']
        start_time = VideoData2.index[start_idx] if isinstance(VideoData2.index, pd.DatetimeIndex) else start_idx
        end_time = VideoData2.index[end_idx] if isinstance(VideoData2.index, pd.DatetimeIndex) else end_idx
        print(f"  Section {i}: index {start_idx}-{end_idx} (length {sec['length']})")



In [ ]:
# center coordinates on median pupil centre, removing outliers and (TODO - low confidence inference points)
############################################################################################################

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

print("=== Centering ===")
# VideoData1 processing
if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
    # Calculate the mean of the center x and y points
    mean_center_x1 = VideoData1['center.x'].median()
    mean_center_y1 = VideoData1['center.y'].median()

    print(f"VideoData1 - Centering on median pupil centre: \nMean center.x: {mean_center_x1}, Mean center.y: {mean_center_y1}")

    # Translate the coordinates
    for col in columns_of_interest:
        if '.x' in col:
            VideoData1[col] = VideoData1[col] - mean_center_x1
        elif '.y' in col:
            VideoData1[col] = VideoData1[col] - mean_center_y1

    VideoData1_centered = VideoData1.copy()

# VideoData2 processing
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    # Calculate the mean of the center x and y points
    mean_center_x2 = VideoData2['center.x'].median()
    mean_center_y2 = VideoData2['center.y'].median()

    print(f"VideoData2 - Centering on median pupil centre: \nMean center.x: {mean_center_x2}, Mean center.y: {mean_center_y2}")

    # Translate the coordinates
    for col in columns_of_interest:
        if '.x' in col:
            VideoData2[col] = VideoData2[col] - mean_center_x2
        elif '.y' in col:
            VideoData2[col] = VideoData2[col] - mean_center_y2

    VideoData2_centered = VideoData2.copy()

############################################################################################################
# remove low confidence points (score < threshold)
############################################################################################################
if not NaNs_removed:
    print("\n=== Score-based Filtering ===")
    print(f"Score threshold: {score_cutoff}")
    # List of point names (without .x, .y, .score)
    point_names = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

    # VideoData1 score-based filtering
    if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
        total_low_score1 = 0
        low_score_counts1 = {}
        for point in point_names:
            if f'{point}.score' in VideoData1.columns:
                # Find indices where score is below threshold
                low_score_mask = VideoData1[f'{point}.score'] < score_cutoff
                low_score_count = low_score_mask.sum()
                low_score_counts1[f'{point}.x'] = low_score_count
                low_score_counts1[f'{point}.y'] = low_score_count
                total_low_score1 += low_score_count * 2  # *2 because we're removing both x and y
                
                # Set x and y to NaN for low confidence points
                VideoData1.loc[low_score_mask, f'{point}.x'] = np.nan
                VideoData1.loc[low_score_mask, f'{point}.y'] = np.nan
        
        # Find the channel with the maximum number of low-score points
        max_low_score_channel1 = max(low_score_counts1, key=low_score_counts1.get)
        max_low_score_count1 = low_score_counts1[max_low_score_channel1]
        
        # Print the channel with the maximum number of low-score points
        print(f"VideoData1 - Channel with the maximum number of low-confidence points: {max_low_score_channel1}, Number of low-confidence points: {max_low_score_count1}")
        print(f"VideoData1 - A total number of {total_low_score1} low-confidence coordinate values were replaced by interpolation")

    # VideoData2 score-based filtering
    if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
        total_low_score2 = 0
        low_score_counts2 = {}
        for point in point_names:
            if f'{point}.score' in VideoData2.columns:
                # Find indices where score is below threshold
                low_score_mask = VideoData2[f'{point}.score'] < score_cutoff
                low_score_count = low_score_mask.sum()
                low_score_counts2[f'{point}.x'] = low_score_count
                low_score_counts2[f'{point}.y'] = low_score_count
                total_low_score2 += low_score_count * 2  # *2 because we're removing both x and y
                
                # Set x and y to NaN for low confidence points
                VideoData2.loc[low_score_mask, f'{point}.x'] = np.nan
                VideoData2.loc[low_score_mask, f'{point}.y'] = np.nan
        
        # Find the channel with the maximum number of low-score points
        max_low_score_channel2 = max(low_score_counts2, key=low_score_counts2.get)
        max_low_score_count2 = low_score_counts2[max_low_score_channel2]
        
        # Print the channel with the maximum number of low-score points
        print(f"VideoData2 - Channel with the maximum number of low-confidence points: {max_low_score_channel2}, Number of low-confidence points: {max_low_score_count2}")
        print(f"VideoData2 - A total number of {total_low_score2} low-confidence coordinate values were replaced by interpolation")

    ############################################################################################################
    # remove outliers (x times SD)
    # then interpolates on all NaN values (skipped frames, low confidence inference points, outliers)
    ############################################################################################################

    print("\n=== Outlier Analysis ===")

    # VideoData1 outlier analysis and interpolation
    if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
        # Calculate the standard deviation for each column of interest
        std_devs1 = {col: VideoData1[col].std() for col in columns_of_interest}

        # Calculate the number of outliers for each column
        outliers1 = {col: ((VideoData1[col] - VideoData1[col].mean()).abs() > 10 * std_devs1[col]).sum() for col in columns_of_interest}

        # Find the channel with the maximum number of outliers
        max_outliers_channel1 = max(outliers1, key=outliers1.get)
        max_outliers_count1 = outliers1[max_outliers_channel1]

        # Print the channel with the maximum number of outliers and the number
        print(f"VideoData1 - Channel with the maximum number of outliers: {max_outliers_channel1}, Number of outliers: {max_outliers_count1}")

        # Print the total number of outliers
        total_outliers1 = sum(outliers1.values())
        print(f"VideoData1 - A total number of {total_outliers1} outliers were replaced by interpolation")

        # Replace outliers by interpolating between the previous and subsequent non-NaN value
        for col in columns_of_interest:
            outlier_indices = VideoData1[((VideoData1[col] - VideoData1[col].mean()).abs() > outlier_sd_threshold * std_devs1[col])].index
            VideoData1.loc[outlier_indices, col] = np.nan

        #VideoData1.interpolate(inplace=True)
        VideoData1 = VideoData1.interpolate(method='linear', limit_direction='both')

    # VideoData2 outlier analysis and interpolation
    if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
        # Calculate the standard deviation for each column of interest
        std_devs2 = {col: VideoData2[col].std() for col in columns_of_interest}

        # Calculate the number of outliers for each column
        outliers2 = {col: ((VideoData2[col] - VideoData2[col].mean()).abs() > 10 * std_devs2[col]).sum() for col in columns_of_interest}

        # Find the channel with the maximum number of outliers
        max_outliers_channel2 = max(outliers2, key=outliers2.get)
        max_outliers_count2 = outliers2[max_outliers_channel2]

        # Print the channel with the maximum number of outliers and the number
        print(f"VideoData2 - Channel with the maximum number of outliers: {max_outliers_channel2}, Number of outliers: {max_outliers_count2}")

        # Print the total number of outliers
        total_outliers2 = sum(outliers2.values())
        print(f"VideoData2 - A total number of {total_outliers2} outliers were replaced by interpolation")

        # Replace outliers by interpolating between the previous and subsequent non-NaN value
        for col in columns_of_interest:
            outlier_indices = VideoData2[((VideoData2[col] - VideoData2[col].mean()).abs() > outlier_sd_threshold * std_devs2[col])].index
            VideoData2.loc[outlier_indices, col] = np.nan

        #VideoData2.interpolate(inplace=True)
        VideoData2 = VideoData2.interpolate(method='linear', limit_direction='both')
        
        NaNs_removed = True
else:
    print("=== Interpolation already done, skipping ===")



In [ ]:
# QC plot timeseries of interpolation corrected NaN and (TODO low confidence coordinates in browser 
############################################################################################################

if plot_timeseries:
    print(f'ℹ️ Figure opens in browser window, takes a bit of time.')
    
    # VideoData1 QC Plot
    if 'VideoData1_Has_Sleap' in globals() and VideoData1_Has_Sleap:
        fig1 = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=(
                "VideoData1 - X coordinates for pupil centre and left-right eye corner",
                "VideoData1 - Y coordinates for pupil centre and left-right eye corner",
                "VideoData1 - X coordinates for iris points",
                "VideoData1 - Y coordinates for iris points"
            )
        )

        # Row 1: Plot left.x, center.x, right.x
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['left.x'], mode='lines', name='left.x'), row=1, col=1)
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['center.x'], mode='lines', name='center.x'), row=1, col=1)
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['right.x'], mode='lines', name='right.x'), row=1, col=1)

        # Row 2: Plot left.y, center.y, right.y
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['left.y'], mode='lines', name='left.y'), row=2, col=1)
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['center.y'], mode='lines', name='center.y'), row=2, col=1)
        fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['right.y'], mode='lines', name='right.y'), row=2, col=1)

        # Row 3: Plot p.x coordinates for p1 to p8
        for col in ['p1.x', 'p2.x', 'p3.x', 'p4.x', 'p5.x', 'p6.x', 'p7.x', 'p8.x']:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=3, col=1)

        # Row 4: Plot p.y coordinates for p1 to p8
        for col in ['p1.y', 'p2.y', 'p3.y', 'p4.y', 'p5.y', 'p6.y', 'p7.y', 'p8.y']:
            fig1.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col), row=4, col=1)

        fig1.update_layout(
            height=1200,
            title_text="VideoData1 - Time series subplots for coordinates (QC after interpolation)",
            showlegend=True
        )
        fig1.update_xaxes(title_text="Seconds", row=4, col=1)
        fig1.update_yaxes(title_text="X Position", row=1, col=1)
        fig1.update_yaxes(title_text="Y Position", row=2, col=1)
        fig1.update_yaxes(title_text="X Position", row=3, col=1)
        fig1.update_yaxes(title_text="Y Position", row=4, col=1)

        fig1.show(renderer='browser')
    
    # VideoData2 QC Plot
    if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
        fig2 = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=(
                "VideoData2 - X coordinates for pupil centre and left-right eye corner",
                "VideoData2 - Y coordinates for pupil centre and left-right eye corner",
                "VideoData2 - X coordinates for iris points",
                "VideoData2 - Y coordinates for iris points"
            )
        )

        # Row 1: Plot left.x, center.x, right.x
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['left.x'], mode='lines', name='left.x'), row=1, col=1)
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['center.x'], mode='lines', name='center.x'), row=1, col=1)
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['right.x'], mode='lines', name='right.x'), row=1, col=1)

        # Row 2: Plot left.y, center.y, right.y
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['left.y'], mode='lines', name='left.y'), row=2, col=1)
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['center.y'], mode='lines', name='center.y'), row=2, col=1)
        fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2['right.y'], mode='lines', name='right.y'), row=2, col=1)

        # Row 3: Plot p.x coordinates for p1 to p8
        for col in ['p1.x', 'p2.x', 'p3.x', 'p4.x', 'p5.x', 'p6.x', 'p7.x', 'p8.x']:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=3, col=1)

        # Row 4: Plot p.y coordinates for p1 to p8
        for col in ['p1.y', 'p2.y', 'p3.y', 'p4.y', 'p5.y', 'p6.y', 'p7.y', 'p8.y']:
            fig2.add_trace(go.Scatter(x=VideoData2['Seconds'], y=VideoData2[col], mode='lines', name=col), row=4, col=1)

        fig2.update_layout(
            height=1200,
            title_text="VideoData2 - Time series subplots for coordinates (QC after interpolation)",
            showlegend=True
        )
        fig2.update_xaxes(title_text="Seconds", row=4, col=1)
        fig2.update_yaxes(title_text="X Position", row=1, col=1)
        fig2.update_yaxes(title_text="Y Position", row=2, col=1)
        fig2.update_yaxes(title_text="X Position", row=3, col=1)
        fig2.update_yaxes(title_text="Y Position", row=4, col=1)

        fig2.show(renderer='browser')


In [ ]:
# QC plot XY coordinate distributions after NaN and ( TODO - low confidence inference points) are interpolated 
##############################################################################################################

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

# Create coordinates_dict for both datasets
if VideoData1_Has_Sleap:
    coordinates_dict1_processed = lp.get_coordinates_dict(VideoData1, columns_of_interest)
if VideoData2_Has_Sleap:
    coordinates_dict2_processed = lp.get_coordinates_dict(VideoData2, columns_of_interest)

columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

# Filter out NaN values and calculate the min and max values for X and Y coordinates for both dict1 and dict2
def min_max_dict(coordinates_dict):
    x_min = min([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].min() for col in columns_of_interest])
    x_max = max([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].max() for col in columns_of_interest])
    y_min = min([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].min() for col in columns_of_interest])
    y_max = max([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].max() for col in columns_of_interest])
    return x_min, x_max, y_min, y_max

if VideoData1_Has_Sleap:
    x_min1, x_max1, y_min1, y_max1 = min_max_dict(coordinates_dict1_processed)
if VideoData2_Has_Sleap:
    x_min2, x_max2, y_min2, y_max2 = min_max_dict(coordinates_dict2_processed)

# Use global min and max for consistency across subplots
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    x_min = min(x_min1, x_min2)
    x_max = max(x_max1, x_max2)
    y_min = min(y_min1, y_min2)
    y_max = max(y_max1, y_max2)
elif VideoData1_Has_Sleap:
    x_min, x_max, y_min, y_max = x_min1, x_max1, y_min1, y_max1
elif VideoData2_Has_Sleap:
    x_min, x_max, y_min, y_max = x_min2, x_max2, y_min2, y_max2

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18, 12))
fig.suptitle('XY coordinate distribution of different points for VideoData1 and VideoData2 post outlier removal and NaN interpolation)', fontsize=14)

# Define colormap for p1-p8
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange']

# Panel 1: left, right, center (VideoData1)
if VideoData1_Has_Sleap:
    ax[0, 0].set_title('VideoData1: left, right, center')
    ax[0, 0].scatter(coordinates_dict1_processed['left.x'], coordinates_dict1_processed['left.y'], color='black', label='left', s=10)
    ax[0, 0].scatter(coordinates_dict1_processed['right.x'], coordinates_dict1_processed['right.y'], color='grey', label='right', s=10)
    ax[0, 0].scatter(coordinates_dict1_processed['center.x'], coordinates_dict1_processed['center.y'], color='red', label='center', s=10)
    ax[0, 0].set_xlim([x_min, x_max])
    ax[0, 0].set_ylim([y_min, y_max])
    ax[0, 0].set_xlabel('x coordinates (pixels)')
    ax[0, 0].set_ylabel('y coordinates (pixels)')
    ax[0, 0].legend(loc='upper right')

    # Panel 2: p1 to p8 (VideoData1)
    ax[0, 1].set_title('VideoData1: p1 to p8')
    for idx, col in enumerate(columns_of_interest[3:]):
        ax[0, 1].scatter(coordinates_dict1_processed[f'{col}.x'], coordinates_dict1_processed[f'{col}.y'], color=colors[idx], label=col, s=5)
    ax[0, 1].set_xlim([x_min, x_max])
    ax[0, 1].set_ylim([y_min, y_max])
    ax[0, 1].set_xlabel('x coordinates (pixels)')
    ax[0, 1].set_ylabel('y coordinates (pixels)')
    ax[0, 1].legend(loc='upper right')

# Panel 3: left, right, center (VideoData2)
if VideoData2_Has_Sleap:
    ax[1, 0].set_title('VideoData2: left, right, center')
    ax[1, 0].scatter(coordinates_dict2_processed['left.x'], coordinates_dict2_processed['left.y'], color='black', label='left', s=10)
    ax[1, 0].scatter(coordinates_dict2_processed['right.x'], coordinates_dict2_processed['right.y'], color='grey', label='right', s=10)
    ax[1, 0].scatter(coordinates_dict2_processed['center.x'], coordinates_dict2_processed['center.y'], color='red', label='center', s=10)
    ax[1, 0].set_xlim([x_min, x_max])
    ax[1, 0].set_ylim([y_min, y_max])
    ax[1, 0].set_xlabel('x coordinates (pixels)')
    ax[1, 0].set_ylabel('y coordinates (pixels)')
    ax[1, 0].legend(loc='upper right')

    # Panel 4: p1 to p8 (VideoData2)
    ax[1, 1].set_title('VideoData2: p1 to p8')
    for idx, col in enumerate(columns_of_interest[3:]):
        ax[1, 1].scatter(coordinates_dict2_processed[f'{col}.x'], coordinates_dict2_processed[f'{col}.y'], color=colors[idx], label=col, s=5)
    ax[1, 1].set_xlim([x_min, x_max])
    ax[1, 1].set_ylim([y_min, y_max])
    ax[1, 1].set_xlabel('x coordinates (pixels)')
    ax[1, 1].set_ylabel('y coordinates (pixels)')
    ax[1, 1].legend(loc='upper right')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()

In [ ]:
# fit ellipses on the 8 points to determine pupil centre and diameter
############################################################################################################

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

# VideoData1 processing
if VideoData1_Has_Sleap:
    print("=== VideoData1 Ellipse Fitting for Pupil Diameter ===")
    coordinates_dict1_processed = lp.get_coordinates_dict(VideoData1, columns_of_interest)

    theta1 = lp.find_horizontal_axis_angle(VideoData1, 'left', 'center')
    center_point1 = lp.get_left_right_center_point(coordinates_dict1_processed)

    columns_of_interest_reformatted = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
    remformatted_coordinates_dict1 = lp.get_reformatted_coordinates_dict(coordinates_dict1_processed, columns_of_interest_reformatted)
    centered_coordinates_dict1 = lp.get_centered_coordinates_dict(remformatted_coordinates_dict1, center_point1)
    rotated_coordinates_dict1 = lp.get_rotated_coordinates_dict(centered_coordinates_dict1, theta1)

    columns_of_interest_ellipse = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
    ellipse_parameters_data1, ellipse_center_points_data1 = lp.get_fitted_ellipse_parameters(rotated_coordinates_dict1, columns_of_interest_ellipse)

    average_diameter1 = np.mean([ellipse_parameters_data1[:,0], ellipse_parameters_data1[:,1]], axis=0)

    SleapVideoData1 = process.convert_arrays_to_dataframe(['Seconds', 'Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'], [VideoData1['Seconds'].values, average_diameter1, ellipse_parameters_data1[:,2], ellipse_center_points_data1[:,0], ellipse_center_points_data1[:,1]])

# VideoData2 processing
if VideoData2_Has_Sleap:
    print("=== VideoData2 Ellipse Fitting for Pupil Diameter ===")
    coordinates_dict2_processed = lp.get_coordinates_dict(VideoData2, columns_of_interest)

    theta2 = lp.find_horizontal_axis_angle(VideoData2, 'left', 'center')
    center_point2 = lp.get_left_right_center_point(coordinates_dict2_processed)

    columns_of_interest_reformatted = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
    remformatted_coordinates_dict2 = lp.get_reformatted_coordinates_dict(coordinates_dict2_processed, columns_of_interest_reformatted)
    centered_coordinates_dict2 = lp.get_centered_coordinates_dict(remformatted_coordinates_dict2, center_point2)
    rotated_coordinates_dict2 = lp.get_rotated_coordinates_dict(centered_coordinates_dict2, theta2)

    columns_of_interest_ellipse = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
    ellipse_parameters_data2, ellipse_center_points_data2 = lp.get_fitted_ellipse_parameters(rotated_coordinates_dict2, columns_of_interest_ellipse)

    average_diameter2 = np.mean([ellipse_parameters_data2[:,0], ellipse_parameters_data2[:,1]], axis=0)

    SleapVideoData2 = process.convert_arrays_to_dataframe(['Seconds', 'Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'], [VideoData2['Seconds'].values, average_diameter2, ellipse_parameters_data2[:,2], ellipse_center_points_data2[:,0], ellipse_center_points_data2[:,1]])

############################################################################################################
# some aggressive filtering of the pupil diameter
############################################################################################################

# VideoData1 filtering
if VideoData1_Has_Sleap:
    print("\n=== VideoData1 Filtering ===")
    # Butterworth filter parameters
    fs1 = 1 / np.median(np.diff(SleapVideoData1['Seconds']))  # Sampling frequency (Hz)
    order = 6

    b1, a1 = butter(order, cutoff / (0.5 * fs1), btype='low')
    SleapVideoData1['Ellipse.Diameter.Filt'] = filtfilt(b1, a1, SleapVideoData1['Ellipse.Diameter'])

    SleapVideoData1['Ellipse.Diameter'] = SleapVideoData1['Ellipse.Diameter'].rolling(window=12, center=True, min_periods=1).median()

# VideoData2 filtering
if VideoData2_Has_Sleap:
    print("=== VideoData2 Filtering ===")
    fs2 = 1 / np.median(np.diff(SleapVideoData2['Seconds']))  # Sampling frequency (Hz)
    order = 6

    b2, a2 = butter(order, cutoff / (0.5 * fs2), btype='low')
    SleapVideoData2['Ellipse.Diameter.Filt'] = filtfilt(b2, a2, SleapVideoData2['Ellipse.Diameter'])

    SleapVideoData2['Ellipse.Diameter'] = SleapVideoData2['Ellipse.Diameter'].rolling(window=12, center=True, min_periods=1).median()

print("✅ Done calculating pupil diameter and angle for both VideoData1 and VideoData2")

In [ ]:
# cross-correlate pupil diameter for left and right eye 
############################################################################################################

if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    # Create subplots for both comparison and cross-correlation
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=["Pupil Diameter Comparison", "Cross-Correlation Analysis"],
        vertical_spacing=0.15
    )

    # Add SleapVideoData1 pupil diameter
    fig.add_trace(
        go.Scatter(
            x=SleapVideoData1['Seconds'],
            y=SleapVideoData1['Ellipse.Diameter'],
            mode='lines',
            name="VideoData1 Pupil Diameter",
            line=dict(color='blue')
        ),
        row=1, col=1
    )

    # Add SleapVideoData2 pupil diameter
    fig.add_trace(
        go.Scatter(
            x=SleapVideoData2['Seconds'],
            y=SleapVideoData2['Ellipse.Diameter'],
            mode='lines',
            name="VideoData2 Pupil Diameter",
            line=dict(color='red')
        ),
        row=1, col=1
    )

    # Cross-correlation analysis
    print("=== Cross-Correlation Analysis ===")

    # Get pupil diameter data
    pupil1 = SleapVideoData1['Ellipse.Diameter'].values
    pupil2 = SleapVideoData2['Ellipse.Diameter'].values

    # Handle different lengths by using the shorter dataset length
    min_length = min(len(pupil1), len(pupil2))

    # Truncate both datasets to the same length
    pupil1_truncated = pupil1[:min_length]
    pupil2_truncated = pupil2[:min_length]

    # Remove NaN values for correlation, shouldn't be necessary as we filtered NaNs out before
    valid_mask1 = ~np.isnan(pupil1_truncated)
    valid_mask2 = ~np.isnan(pupil2_truncated)
    valid_mask = valid_mask1 & valid_mask2

    pupil1_clean = pupil1_truncated[valid_mask]
    pupil2_clean = pupil2_truncated[valid_mask]

    # Check if we have enough data
    if len(pupil1_clean) < 2 or len(pupil2_clean) < 2:
        print("❌ Error: Not enough valid data points for correlation analysis")
    else:
        # Calculate cross-correlation
        try:
            correlation = correlate(pupil1_clean, pupil2_clean, mode='full')
            
            # Calculate lags (in samples)
            lags = np.arange(-len(pupil2_clean) + 1, len(pupil1_clean))
            
            # Convert lags to time (assuming same sampling rate)
            dt = np.median(np.diff(SleapVideoData1['Seconds']))
            lag_times = lags * dt
            
            # Find peak correlation and corresponding lag
            peak_idx = np.argmax(correlation)
            peak_correlation = correlation[peak_idx]
            peak_lag_samples = lags[peak_idx]
            peak_lag_time = lag_times[peak_idx]
            peak_lag_time_display = peak_lag_time # for final QC figure 
            
            print(f"Peak lag (time): {peak_lag_time:.4f} seconds")

        
            # Normalize correlation to [-1, 1] range
            norm_factor = np.sqrt(np.sum(pupil1_clean**2) * np.sum(pupil2_clean**2))
            if norm_factor > 0:
                correlation_normalized = correlation / norm_factor
                peak_correlation_normalized = correlation_normalized[peak_idx]
                print(f"Peak normalized correlation: {peak_correlation_normalized:.4f}")
            else:
                print("❌ Error: Cannot normalize correlation (zero variance)")
                correlation_normalized = correlation
                peak_correlation_normalized = 0
            
            # Plot cross-correlation
            fig.add_trace(
                go.Scatter(
                    x=lag_times,
                    y=correlation_normalized,
                    mode='lines',
                    name="Cross-Correlation",
                    line=dict(color='green')
                ),
                row=2, col=1
            )
            
            # Add vertical line at peak
            fig.add_vline(
                x=peak_lag_time,
                line_dash="dash",
                line_color="red",
                annotation_text=f"Peak: {peak_correlation_normalized:.3f}",
                row=2, col=1
            )
            
        except Exception as e:
            print(f"❌ Error in cross-correlation calculation: {e}")
            # Add empty trace to maintain plot structure
            fig.add_trace(
                go.Scatter(
                    x=[0], y=[0],
                    mode='lines',
                    name="Cross-Correlation (Error)",
                    line=dict(color='gray')
                ),
                row=2, col=1
            )

    # Update axes labels
    fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
    fig.update_yaxes(title_text="Pupil Diameter", row=1, col=1)
    fig.update_xaxes(title_text="Lag (seconds)", row=2, col=1)
    fig.update_yaxes(title_text="Normalized Correlation", row=2, col=1)

    fig.update_layout(
        height=800,
        width=1000,
        title_text="SLEAP Pupil Diameter Analysis: Comparison & Cross-Correlation"
    )

    fig.show()

    # Additional correlation statistics
    if len(pupil1_clean) >= 2 and len(pupil2_clean) >= 2:
        try:
            # Calculate Pearson correlation coefficient
            pearson_r, pearson_p = pearsonr(pupil1_clean, pupil2_clean)
            pearson_r_display = pearson_r
            pearson_p_display = pearson_p
            
            print(f"\n=== Additional Statistics ===")
            print(f"Pearson correlation coefficient: {pearson_r:.2f}")

            # Handle extremely small p-values
            if pearson_p < 1e-300:
                print(f'Pearson p-value: < 1e-300 (extremely significant)')
            else:
                print(f'Pearson p-value: {pearson_p:.5e}')
            
        except Exception as e:
            print(f"❌ Error in additional statistics: {e}")
            pearson_r_display = None
            pearson_p_display = None
    else:
        print("❌ Cannot calculate additional statistics - insufficient data")
else:
    print("Only one eye is present, no pupil diameter cross-correlation can be done")

In [ ]:
# check if Second values match 1:1 between VideoData and SleapVideoData then merge them into VideoData
############################################################################################################

if VideoData1_Has_Sleap is True:
    if VideoData1['Seconds'].equals(SleapVideoData1['Seconds']) is False:
        print("❗ Video1: The 'Seconds' columns DO NOT correspond 1:1 between the two DataFrames. This should not happen")
    else:
        VideoData1 = VideoData1.merge(SleapVideoData1, on='Seconds', how='outer')
        del SleapVideoData1

if VideoData2_Has_Sleap is True:
    if VideoData2['Seconds'].equals(SleapVideoData2['Seconds']) is False:
        print("❗ Video2: The 'Seconds' columns DO NOT correspond 1:1 between the two DataFrames. This should not happen")
    else:
        VideoData2 = VideoData2.merge(SleapVideoData2, on='Seconds', how='outer')
        del SleapVideoData2
gc.collect()
None

In [ ]:
# Compare SLEAP center.x and .y with fitted ellipse centre distributions for both VideoData1 and VideoData2
############################################################################################################

# ------------------------------------------------------------------
# 1) Compute correlations for VideoData1
# ------------------------------------------------------------------
if VideoData1_Has_Sleap is True:
    print("=== VideoData1 Analysis ===")
    slope_x1, intercept_x1, r_value_x1, p_value_x1, std_err_x1 = linregress(
        VideoData1["Ellipse.Center.X"], 
        VideoData1["center.x"]
    )
    r_squared_x1 = r_value_x1**2
    print(f"VideoData1 - R^2 between center point and ellipse center X data: {r_squared_x1:.4f}")

    slope_y1, intercept_y1, r_value_y1, p_value_y1, std_err_y1 = linregress(
        VideoData1["Ellipse.Center.Y"], 
        VideoData1["center.y"]
    )
    r_squared_y1 = r_value_y1**2
    print(f"VideoData1 - R^2 between center point and ellipse center Y data: {r_squared_y1:.4f}")

# ------------------------------------------------------------------
# 2) Compute correlations for VideoData2
# ------------------------------------------------------------------
if VideoData2_Has_Sleap is True:
    print("\n=== VideoData2 Analysis ===")
    slope_x2, intercept_x2, r_value_x2, p_value_x2, std_err_x2 = linregress(
        VideoData2["Ellipse.Center.X"], 
        VideoData2["center.x"]
    )
    r_squared_x2 = r_value_x2**2
    print(f"VideoData2 - R^2 between center point and ellipse center X data: {r_squared_x2:.4f}")

    slope_y2, intercept_y2, r_value_y2, p_value_y2, std_err_y2 = linregress(
        VideoData2["Ellipse.Center.Y"], 
        VideoData2["center.y"]
    )
    r_squared_y2 = r_value_y2**2
    print(f"VideoData2 - R^2 between center point and ellipse center Y data: {r_squared_y2:.4f}")

# ------------------------------------------------------------------
# 3) Center of Mass Analysis (if both VideoData1 and VideoData2 are available)
# ------------------------------------------------------------------
if VideoData1_Has_Sleap is True and VideoData2_Has_Sleap is True:
    print("\n=== Center of Mass Distance Analysis ===")
    
    # Calculate center of mass (mean) for VideoData1
    com_center_x1 = VideoData1["center.x"].mean()
    com_center_y1 = VideoData1["center.y"].mean()
    com_ellipse_x1 = VideoData1["Ellipse.Center.X"].mean()
    com_ellipse_y1 = VideoData1["Ellipse.Center.Y"].mean()
    
    # Calculate absolute distances for VideoData1
    dist_x1 = abs(com_center_x1 - com_ellipse_x1)
    dist_y1 = abs(com_center_y1 - com_ellipse_y1)
    
    print(f"\nVideoData1:")
    print(f"  Center of mass for center.x/y: ({com_center_x1:.4f}, {com_center_y1:.4f})")
    print(f"  Center of mass for Ellipse.Center.X/Y: ({com_ellipse_x1:.4f}, {com_ellipse_y1:.4f})")
    print(f"  Absolute distance in X: {dist_x1:.4f} pixels")
    print(f"  Absolute distance in Y: {dist_y1:.4f} pixels")
    
    # Calculate center of mass (mean) for VideoData2
    com_center_x2 = VideoData2["center.x"].mean()
    com_center_y2 = VideoData2["center.y"].mean()
    com_ellipse_x2 = VideoData2["Ellipse.Center.X"].mean()
    com_ellipse_y2 = VideoData2["Ellipse.Center.Y"].mean()
    
    # Calculate absolute distances for VideoData2
    dist_x2 = abs(com_center_x2 - com_ellipse_x2)
    dist_y2 = abs(com_center_y2 - com_ellipse_y2)
    
    print(f"\nVideoData2:")
    print(f"  Center of mass for center.x/y: ({com_center_x2:.4f}, {com_center_y2:.4f})")
    print(f"  Center of mass for Ellipse.Center.X/Y: ({com_ellipse_x2:.4f}, {com_ellipse_y2:.4f})")
    print(f"  Absolute distance in X: {dist_x2:.4f} pixels")
    print(f"  Absolute distance in Y: {dist_y2:.4f} pixels")

# ------------------------------------------------------------------
# 4) Re-center Ellipse.Center.X and Ellipse.Center.Y using median
# ------------------------------------------------------------------
print("\n=== Re-centering Ellipse.Center coordinates ===")

# Re-center VideoData1 Ellipse.Center coordinates
if VideoData1_Has_Sleap is True:
    # Calculate median
    median_ellipse_x1 = VideoData1["Ellipse.Center.X"].median()
    median_ellipse_y1 = VideoData1["Ellipse.Center.Y"].median()
    
    # Center the coordinates
    VideoData1["Ellipse.Center.X"] = VideoData1["Ellipse.Center.X"] - median_ellipse_x1
    VideoData1["Ellipse.Center.Y"] = VideoData1["Ellipse.Center.Y"] - median_ellipse_y1
    
    print(f"VideoData1 - Re-centered Ellipse.Center using median: ({median_ellipse_x1:.4f}, {median_ellipse_y1:.4f})")

# Re-center VideoData2 Ellipse.Center coordinates
if VideoData2_Has_Sleap is True:
    # Calculate median
    median_ellipse_x2 = VideoData2["Ellipse.Center.X"].median()
    median_ellipse_y2 = VideoData2["Ellipse.Center.Y"].median()
    
    # Center the coordinates
    VideoData2["Ellipse.Center.X"] = VideoData2["Ellipse.Center.X"] - median_ellipse_x2
    VideoData2["Ellipse.Center.Y"] = VideoData2["Ellipse.Center.Y"] - median_ellipse_y2
    
    print(f"VideoData2 - Re-centered Ellipse.Center using median: ({median_ellipse_x2:.4f}, {median_ellipse_y2:.4f})")



In [ ]:
# Make and save summary QC plot using matplotlib with scatter plots for 2D distributions

# Initialize the statistics variables (these are calculated in Cell 11)
try:
    pearson_r_display
except NameError:
    pearson_r_display = None
    pearson_p_display = None
    peak_lag_time_display = None
    print("⚠️ Note: Statistics not found. They should be calculated in Cell 11.")

# Calculate correlation for Ellipse.Center.X between VideoData1 and VideoData2 (if both exist)
pearson_r_center = None
pearson_p_center = None
peak_lag_time_center = None

if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    # Get the Center.X data
    center_x1 = VideoData1['Ellipse.Center.X'].values
    center_x2 = VideoData2['Ellipse.Center.X'].values
    
    min_length = min(len(center_x1), len(center_x2))
    center_x1_truncated = center_x1[:min_length]
    center_x2_truncated = center_x2[:min_length]
    
    valid_mask1 = ~np.isnan(center_x1_truncated)
    valid_mask2 = ~np.isnan(center_x2_truncated)
    valid_mask = valid_mask1 & valid_mask2
    
    center_x1_clean = center_x1_truncated[valid_mask]
    center_x2_clean = center_x2_truncated[valid_mask]
    
    if len(center_x1_clean) >= 2 and len(center_x2_clean) >= 2:
        try:
            # Calculate Pearson correlation
            pearson_r_center, pearson_p_center = pearsonr(center_x1_clean, center_x2_clean)
            
            # Calculate cross-correlation for peak lag
            correlation = correlate(center_x1_clean, center_x2_clean, mode='full')
            lags = np.arange(-len(center_x2_clean) + 1, len(center_x1_clean))
            dt = np.median(np.diff(VideoData1['Seconds']))
            lag_times = lags * dt
            peak_idx = np.argmax(correlation)
            peak_lag_time_center = lag_times[peak_idx]
        except Exception as e:
            print(f"❌ Error calculating Ellipse.Center.X correlation stats: {e}")

# Create the QC summary figure using matplotlib with custom grid layout
fig = plt.figure(figsize=(20, 18))
fig.suptitle(str(data_path), fontsize=16, y=0.995)

# Create a grid layout:
# - Top row (full width): VideoData1 Time Series
# - Second row (full width): VideoData2 Time Series  
# - Third row (two columns): 2D scatter plots (VideoData1 left, VideoData2 right)
# - Fourth row (two columns): Pupil diameter (left), Ellipse.Center.X correlation (right)

gs = fig.add_gridspec(4, 2, hspace=0.3, wspace=0.3)

# Panel 1: VideoData1 center coordinates - Time Series (full width)
if VideoData1_Has_Sleap:
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(VideoData1_centered['Seconds'], VideoData1_centered['center.x'],
            linewidth=0.5, c='blue', alpha=0.6, label='center.x original')
    ax1.plot(VideoData1['Seconds'], VideoData1['Ellipse.Center.X'],
            linewidth=0.5, c='red', alpha=0.6, label='Ellipse Center.X')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Position (pixels)')
    ax1.set_title('VideoData1 - center.X Time Series')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

# Panel 2: VideoData2 center coordinates - Time Series (full width)
if VideoData2_Has_Sleap:
    ax2 = fig.add_subplot(gs[1, :])
    ax2.plot(VideoData2_centered['Seconds'], VideoData2_centered['center.x'],
            linewidth=0.5, c='blue', alpha=0.6, label='center.x original')
    ax2.plot(VideoData2['Seconds'], VideoData2['Ellipse.Center.X'],
            linewidth=0.5, c='red', alpha=0.6, label='Ellipse Center.X')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Position (pixels)')
    ax2.set_title('VideoData2 - center.X Time Series')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# Panel 3: VideoData1 center coordinates - Scatter plot (left half)
if VideoData1_Has_Sleap:
    ax3 = fig.add_subplot(gs[2, 0])
    
    # Ellipse.Center (blue)
    x_ellipse1 = VideoData1['Ellipse.Center.X'].to_numpy()
    y_ellipse1 = VideoData1['Ellipse.Center.Y'].to_numpy()
    mask1 = ~(np.isnan(x_ellipse1) | np.isnan(y_ellipse1))
    
    ax3.scatter(x_ellipse1[mask1], y_ellipse1[mask1],
               s=1, alpha=0.3, c='blue', label='Ellipse.Center')
    
    # Center (red) - from centered data
    x_center1 = VideoData1_centered['center.x'].to_numpy()
    y_center1 = VideoData1_centered['center.y'].to_numpy()
    mask2 = ~(np.isnan(x_center1) | np.isnan(y_center1))
    
    ax3.scatter(x_center1[mask2], y_center1[mask2],
               s=1, alpha=0.3, c='red', label='center.x original')
    
    ax3.set_xlabel('Center X (pixels)')
    ax3.set_ylabel('Center Y (pixels)')
    ax3.set_title('VideoData1 - Center X-Y Distribution (center.X vs Ellipse)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Add R² statistics for VideoData1 (bottom left)
    try:
        if 'r_squared_x1' in globals() and 'r_squared_y1' in globals():
            stats_text = f'R² X: {r_squared_x1:.2g}\nR² Y: {r_squared_y1:.2g}'
            ax3.text(0.02, 0.02, stats_text, transform=ax3.transAxes,
                    verticalalignment='bottom', horizontalalignment='left',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                    fontsize=9, family='monospace')
    except:
        pass
    
    # Add center of mass distance for VideoData1 (bottom right)
    try:
        if 'dist_x1' in globals() and 'dist_y1' in globals():
            distance_text = f'COM Dist X: {dist_x1:.3g}\nCOM Dist Y: {dist_y1:.3g}'
            ax3.text(0.98, 0.02, distance_text, transform=ax3.transAxes,
                    verticalalignment='bottom', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                    fontsize=9, family='monospace')
    except:
        pass

# Panel 4: VideoData2 center coordinates - Scatter plot (right half)
if VideoData2_Has_Sleap:
    ax4 = fig.add_subplot(gs[2, 1])
    
    # Ellipse.Center (blue)
    x_ellipse2 = VideoData2['Ellipse.Center.X'].to_numpy()
    y_ellipse2 = VideoData2['Ellipse.Center.Y'].to_numpy()
    mask3 = ~(np.isnan(x_ellipse2) | np.isnan(y_ellipse2))
    
    ax4.scatter(x_ellipse2[mask3], y_ellipse2[mask3],
               s=1, alpha=0.3, c='blue', label='Ellipse.Center')
    
    # Center (red) - from centered data
    x_center2 = VideoData2_centered['center.x'].to_numpy()
    y_center2 = VideoData2_centered['center.y'].to_numpy()
    mask4 = ~(np.isnan(x_center2) | np.isnan(y_center2))
    
    ax4.scatter(x_center2[mask4], y_center2[mask4],
               s=1, alpha=0.3, c='red', label='center.X Center')
    
    ax4.set_xlabel('Center X (pixels)')
    ax4.set_ylabel('Center Y (pixels)')
    ax4.set_title('VideoData2 - Center X-Y Distribution (center.X vs Ellipse)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # Add R² statistics for VideoData2 (bottom left)
    try:
        if 'r_squared_x2' in globals() and 'r_squared_y2' in globals():
            stats_text = f'R² X: {r_squared_x2:.2g}\nR² Y: {r_squared_y2:.2g}'
            ax4.text(0.02, 0.02, stats_text, transform=ax4.transAxes,
                    verticalalignment='bottom', horizontalalignment='left',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                    fontsize=9, family='monospace')
    except:
        pass
    
    # Add center of mass distance for VideoData2 (bottom right)
    try:
        if 'dist_x2' in globals() and 'dist_y2' in globals():
            distance_text = f'COM Dist X: {dist_x2:.3g}\nCOM Dist Y: {dist_y2:.3g}'
            ax4.text(0.98, 0.02, distance_text, transform=ax4.transAxes,
                    verticalalignment='bottom', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                    fontsize=9, family='monospace')
    except:
        pass

# Panel 5: Pupil diameter comparison (bottom left)
ax5 = fig.add_subplot(gs[3, 0])
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    ax5.plot(VideoData1['Seconds'], VideoData1['Ellipse.Diameter.Filt'],
            linewidth=0.5, c='#FF7F00', alpha=0.6, label='VideoData1 Diameter')
    ax5.plot(VideoData2['Seconds'], VideoData2['Ellipse.Diameter.Filt'],
            linewidth=0.5, c='#9370DB', alpha=0.6, label='VideoData2 Diameter')
elif VideoData1_Has_Sleap:
    ax5.plot(VideoData1['Seconds'], VideoData1['Ellipse.Diameter.Filt'],
            linewidth=0.5, c='#FF7F00', alpha=0.6, label='VideoData1 Diameter')
elif VideoData2_Has_Sleap:
    ax5.plot(VideoData2['Seconds'], VideoData2['Ellipse.Diameter.Filt'],
            linewidth=0.5, c='#9370DB', alpha=0.6, label='VideoData2 Diameter')

ax5.set_xlabel('Time (s)')
ax5.set_ylabel('Diameter (pixels)')
ax5.set_title('Pupil Diameter Comparison')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Add statistics text to Panel 5
if pearson_r_display is not None and pearson_p_display is not None and peak_lag_time_display is not None:
    stats_text = (f'Pearson r = {pearson_r_display:.4f}\n'
                  f'Pearson p = {pearson_p_display:.4e}\n'
                  f'Peak lag = {peak_lag_time_display:.4f} s')
    ax5.text(0.98, 0.98, stats_text, transform=ax5.transAxes,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
            fontsize=10, family='monospace')
else:
    ax5.text(0.5, 0.5, 'Statistics not available\n(See Cell 11 for correlation analysis)', 
            transform=ax5.transAxes, ha='center', va='center', fontsize=10)

# Panel 6: Ellipse.Center.X comparison (bottom right) with dual y-axis
ax6 = fig.add_subplot(gs[3, 1])
ax6_twin = ax6.twinx()  # Create a second y-axis

if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    # Plot the individual traces
    ax6.plot(VideoData1['Seconds'], VideoData1['Ellipse.Center.X'],
            linewidth=0.5, c='#FF7F00', alpha=0.6, label='VideoData1 Ellipse.Center.X')
    ax6.plot(VideoData2['Seconds'], VideoData2['Ellipse.Center.X'],
            linewidth=0.5, c='#9370DB', alpha=0.6, label='VideoData2 Ellipse.Center.X')
    
    # Plot the difference on the right axis
    # Align the data to the same length and normalize for fair comparison
    min_length = min(len(VideoData1), len(VideoData2))
    
    # Normalize data (z-score) to account for different scales
    center_x1_aligned = VideoData1['Ellipse.Center.X'].iloc[:min_length]
    center_x2_aligned = VideoData2['Ellipse.Center.X'].iloc[:min_length]
    
    # Calculate mean and std for normalization
    mean1 = center_x1_aligned.mean()
    std1 = center_x1_aligned.std()
    mean2 = center_x2_aligned.mean()
    std2 = center_x2_aligned.std()
    
    # Normalize both datasets
    center_x1_norm = (center_x1_aligned - mean1) / std1
    center_x2_norm = (center_x2_aligned - mean2) / std2
    
    # Calculate difference of normalized data
    center_x_diff = center_x1_norm - center_x2_norm
    seconds_aligned = VideoData1['Seconds'].iloc[:min_length]
    ax6_twin.plot(seconds_aligned, center_x_diff,
                  linewidth=0.5, c='green', alpha=0.6, label='Difference (normalized)')
    
elif VideoData1_Has_Sleap:
    ax6.plot(VideoData1['Seconds'], VideoData1['Ellipse.Center.X'],
            linewidth=0.5, c='#FF7F00', alpha=0.6, label='VideoData1 Ellipse.Center.X')
elif VideoData2_Has_Sleap:
    ax6.plot(VideoData2['Seconds'], VideoData2['Ellipse.Center.X'],
            linewidth=0.5, c='#9370DB', alpha=0.6, label='VideoData2 Ellipse.Center.X')

ax6.set_xlabel('Time (s)')
ax6.set_ylabel('Center X (pixels)', color='black')
ax6.set_title('Ellipse.Center.X Comparison')
ax6.tick_params(axis='y', labelcolor='black')
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    ax6_twin.set_ylabel('Normalized Difference (z-score)', color='green')
    ax6_twin.tick_params(axis='y', labelcolor='green')

# Combine legends from both axes
lines1, labels1 = ax6.get_legend_handles_labels()
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    lines2, labels2 = ax6_twin.get_legend_handles_labels()
    ax6.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
else:
    ax6.legend(loc='upper left')

ax6.grid(True, alpha=0.3)

# Add statistics text to Panel 6
if pearson_r_center is not None and pearson_p_center is not None and peak_lag_time_center is not None:
    stats_text = (f'Pearson r = {pearson_r_center:.4f}\n'
                  f'Pearson p = {pearson_p_center:.4e}\n'
                  f'Peak lag = {peak_lag_time_center:.4f} s')
    ax6.text(0.98, 0.98, stats_text, transform=ax6.transAxes,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
            fontsize=10, family='monospace')
else:
    ax6.text(0.5, 0.5, 'Statistics not available\n(both eyes required)', 
            transform=ax6.transAxes, ha='center', va='center', fontsize=10)

# Save as PDF (editable vector format)
save_path.mkdir(parents=True, exist_ok=True)
pdf_path = save_path / "Eye_data_QC.pdf"
plt.savefig(pdf_path, dpi=300, bbox_inches='tight', format='pdf')
print(f"✅ QC figure saved as PDF (editable): {pdf_path}")

# Also save as 600 dpi PNG (high-resolution for printing)
png_path = save_path / "Eye_data_QC.png"
plt.savefig(png_path, dpi=600, bbox_inches='tight', format='png')
print(f"✅ QC figure saved as PNG (600 dpi for printing): {png_path}")


plt.show()


In [ ]:
# Create interactive time series plots using plotly for browser viewing

# Create subplots for the time series (3 rows now instead of 2)
# Need to enable secondary_y for the third panel
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=('VideoData1 - center.X Time Series', 
                    'VideoData2 - center.X Time Series',
                    'Ellipse.Center.X Comparison with Difference'),
    specs=[[{}], [{}], [{"secondary_y": True}]]  # Enable secondary_y for row 3
)

# Panel 1: VideoData1 center coordinates - Time Series
if VideoData1_Has_Sleap:
    fig.add_trace(go.Scatter(
        x=VideoData1_centered['Seconds'],
        y=VideoData1_centered['center.x'],
        mode='lines',
        name='center.x original',
        line=dict(color='blue', width=0.5),
        opacity=0.6
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=VideoData1['Seconds'],
        y=VideoData1['Ellipse.Center.X'],
        mode='lines',
        name='Ellipse Center.X',
        line=dict(color='red', width=0.5),
        opacity=0.6
    ), row=1, col=1)

# Panel 2: VideoData2 center coordinates - Time Series
if VideoData2_Has_Sleap:
    fig.add_trace(go.Scatter(
        x=VideoData2_centered['Seconds'],
        y=VideoData2_centered['center.x'],
        mode='lines',
        name='center.x original',
        line=dict(color='blue', width=0.5),
        opacity=0.6
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=VideoData2['Seconds'],
        y=VideoData2['Ellipse.Center.X'],
        mode='lines',
        name='Ellipse Center.X',
        line=dict(color='red', width=0.5),
        opacity=0.6
    ), row=2, col=1)

# Panel 3: Ellipse.Center.X Comparison with difference
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    # Plot the individual traces
    fig.add_trace(go.Scatter(
        x=VideoData1['Seconds'],
        y=VideoData1['Ellipse.Center.X'],
        mode='lines',
        name='VideoData1 Ellipse.Center.X',
        line=dict(color='#FF7F00', width=0.5),  # Orange
        opacity=0.6
    ), row=3, col=1)
    
    fig.add_trace(go.Scatter(
        x=VideoData2['Seconds'],
        y=VideoData2['Ellipse.Center.X'],
        mode='lines',
        name='VideoData2 Ellipse.Center.X',
        line=dict(color='#9370DB', width=0.5),  # Purple
        opacity=0.6
    ), row=3, col=1)
    
    # Plot the difference on secondary y-axis
    # Align the data to the same length and normalize for fair comparison
    min_length = min(len(VideoData1), len(VideoData2))
    
    # Normalize data (z-score) to account for different scales
    center_x1_aligned = VideoData1['Ellipse.Center.X'].iloc[:min_length]
    center_x2_aligned = VideoData2['Ellipse.Center.X'].iloc[:min_length]
    
    # Calculate mean and std for normalization
    mean1 = center_x1_aligned.mean()
    std1 = center_x1_aligned.std()
    mean2 = center_x2_aligned.mean()
    std2 = center_x2_aligned.std()
    
    # Normalize both datasets
    center_x1_norm = (center_x1_aligned - mean1) / std1
    center_x2_norm = (center_x2_aligned - mean2) / std2
    
    # Calculate difference of normalized data
    center_x_diff = center_x1_norm - center_x2_norm
    seconds_aligned = VideoData1['Seconds'].iloc[:min_length]
    
    fig.add_trace(go.Scatter(
        x=seconds_aligned,
        y=center_x_diff,
        mode='lines',
        name='Difference (normalized)',
        line=dict(color='green', width=0.5),
        opacity=0.6
    ), row=3, col=1, secondary_y=True)
    
elif VideoData1_Has_Sleap:
    fig.add_trace(go.Scatter(
        x=VideoData1['Seconds'],
        y=VideoData1['Ellipse.Center.X'],
        mode='lines',
        name='VideoData1 Ellipse.Center.X',
        line=dict(color='#FF7F00', width=0.5),
        opacity=0.6
    ), row=3, col=1)
elif VideoData2_Has_Sleap:
    fig.add_trace(go.Scatter(
        x=VideoData2['Seconds'],
        y=VideoData2['Ellipse.Center.X'],
        mode='lines',
        name='VideoData2 Ellipse.Center.X',
        line=dict(color='#9370DB', width=0.5),
        opacity=0.6
    ), row=3, col=1)

# Update layout
fig.update_layout(
    height=1200,  # Increased height for 3 panels
    title_text=f'{data_path} - Eye Tracking Time Series QC',
    showlegend=True,
    hovermode='x unified'
)

# Update axes
fig.update_xaxes(title_text="Time (s)", row=3, col=1)
fig.update_yaxes(title_text="Position (pixels)", row=1, col=1)
fig.update_yaxes(title_text="Position (pixels)", row=2, col=1)
fig.update_yaxes(title_text="Center X (pixels)", row=3, col=1)

# Update secondary y-axis for difference plot
if VideoData1_Has_Sleap and VideoData2_Has_Sleap:
    fig.update_yaxes(title_text="Normalized Difference (z-score)", row=3, col=1, secondary_y=True)

# Show in browser
fig.show(renderer='browser')

# Also save as HTML
save_path.mkdir(parents=True, exist_ok=True)
html_path = save_path / "Eye_data_QC_time_series.html"
fig.write_html(html_path)
print(f"✅ Interactive time series plot saved to: {html_path}")


In [ ]:
# save as df to csv to be loaded in the photometry/harp/etc. analysis notebook 
############################################################################################################
# reindex to aeon datetime to be done in the other notebook
 
if VideoData1_Has_Sleap:
    # Save  DataFrame as CSV to proper path and filename
    save_path1 = save_path / "Video_Sleap_Data1" / "Video_Sleap_Data1_1904-01-01T00-00-00.csv"
    save_path1.parent.mkdir(parents=True, exist_ok=True)
    #save_path1.parent.mkdir(parents=True, exist_ok=True)
    VideoData1.to_csv(save_path1)

if VideoData2_Has_Sleap:
    # Save  DataFrame as CSV to proper path and filename
    save_path2 = save_path / "Video_Sleap_Data2" / "Video_Sleap_Data2_1904-01-01T00-00-00.csv"
    save_path2.parent.mkdir(parents=True, exist_ok=True)
    #save_path2.parent.mkdir(parents=True, exist_ok=True)
    VideoData2.to_csv(save_path2)


# Saccade detection


In [ ]:
### smooth and plot position X and velocity data 

# 1. Preprocess:  smooth
df = VideoData1[["Ellipse.Center.X", "Seconds"]].copy()

df['X_smooth'] = (
    df['Ellipse.Center.X']
      .rolling(window=5, center=True)
      .median()
      .bfill()
      .ffill()
)

# 2. Compute instantaneous velocity for both original and smoothed data
#   dt in seconds
df['dt'] = df['Seconds'].diff()
#   Original velocity
df['vel_x_original'] = df['Ellipse.Center.X'].diff() / df['dt']
#   Smoothed velocity
df['vel_x_smooth'] = df['X_smooth'].diff() / df['dt']

# --- Plot original and smoothed traces with overlay ---
# Create subplots with shared x-axis for synchronized zooming
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,  # This ensures x-axis zoom synchronization
    vertical_spacing=0.1,
    subplot_titles=('X Position (px)', 'Velocity (px/s)')
)

# Add original X position to the first subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['Ellipse.Center.X'],
        mode='lines',
        name='Original X',
        line=dict(color='lightblue', width=2),
        opacity=0.8
    ),
    row=1, col=1
)

# Add smoothed X to the first subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['X_smooth'],
        mode='lines',
        name='Smoothed X',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

# Add original velocity to the second subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['vel_x_original'],
        mode='lines',
        name='Original Velocity',
        line=dict(color='lightcoral', width=2),
        opacity=0.8
    ),
    row=2, col=1
)

# Add smoothed velocity to the second subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['vel_x_smooth'],
        mode='lines',
        name='Smoothed Velocity',
        line=dict(color='red', width=2)
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title='Original vs Smoothed X Position and Velocity (Synchronized Zoom)',
    height=600,  # Adjust height for two subplots
    showlegend=True,
    legend=dict(x=0.01, y=0.99)
)

# Update x-axes
fig.update_xaxes(title_text="Time (s)", row=2, col=1)

# Update y-axes
fig.update_yaxes(title_text="X Position (px)", row=1, col=1)
fig.update_yaxes(title_text="Velocity (px/s)", row=2, col=1)

fig.show()

In [21]:
# SACCADE DETECTION (Adaptive Statistical Method)
#-------------------------------------------------------------------------------
# This approach uses statistical methods to set adaptive thresholds based on the data itself,
# which better captures both large and small saccades

# 1. Calculate adaptive thresholds using statistical methods
abs_vel = df['vel_x_smooth'].abs().dropna()
vel_thresh = abs_vel.mean() + k * abs_vel.std()

print(f"Adaptive velocity threshold: {vel_thresh:.2f} px/s")
print(f"  (mean: {abs_vel.mean():.2f} px/s, std: {abs_vel.std():.2f} px/s, k={k})")
print(f"Saccade duration parameter: {onset_offset_fraction} (saccade ends when velocity < {vel_thresh * onset_offset_fraction:.2f} px/s)")

# 2. Find peaks using scipy's find_peaks with adaptive height
# Find positive peaks (upward saccades)
pos_peaks, pos_properties = find_peaks(
    df['vel_x_smooth'],
    height=vel_thresh,  # Minimum peak height
    distance=int(FPS_1 * refractory_period),  # Minimum distance between peaks (~100ms refractory period)
    width=1  # Minimum peak width in samples
)

# Find negative peaks (downward saccades) by inverting the signal
neg_peaks, neg_properties = find_peaks(
    -df['vel_x_smooth'],  # Invert to find troughs
    height=vel_thresh,  # Same threshold
    distance=int(FPS_1 * refractory_period),
    width=1
)

# 3. Extract saccade information
upward_saccades = []
for peak_idx in pos_peaks:
    peak_time = df.iloc[peak_idx]['Seconds']
    peak_velocity = df.iloc[peak_idx]['vel_x_smooth']
    
    # Optional: find onset and offset by going backward/forward until velocity drops below threshold
    start_idx = peak_idx
    end_idx = peak_idx
    
    # Find onset (go backward)
    while start_idx > 0 and abs(df.iloc[start_idx]['vel_x_smooth']) > vel_thresh * onset_offset_fraction:
        start_idx -= 1
    
    # Find offset (go forward)
    while end_idx < len(df) - 1 and abs(df.iloc[end_idx]['vel_x_smooth']) > vel_thresh * onset_offset_fraction:
        end_idx += 1
    
    onset_time = df.iloc[start_idx]['Seconds']
    offset_time = df.iloc[end_idx]['Seconds']
    
    # Calculate amplitude (absolute change in X position)
    start_x = df.iloc[start_idx]['X_smooth']
    end_x = df.iloc[end_idx]['X_smooth']
    amplitude = abs(end_x - start_x)
    
    upward_saccades.append({
        'time': peak_time,
        'velocity': peak_velocity,
        'start_time': onset_time,
        'end_time': offset_time,
        'duration': offset_time - onset_time,
        'start_position': start_x,
        'end_position': end_x,
        'amplitude': amplitude
    })

downward_saccades = []
for peak_idx in neg_peaks:
    peak_time = df.iloc[peak_idx]['Seconds']
    peak_velocity = df.iloc[peak_idx]['vel_x_smooth']
    
    # Find onset and offset
    start_idx = peak_idx
    end_idx = peak_idx
    
    # Find onset
    while start_idx > 0 and abs(df.iloc[start_idx]['vel_x_smooth']) > vel_thresh * onset_offset_fraction:
        start_idx -= 1
    
    # Find offset
    while end_idx < len(df) - 1 and abs(df.iloc[end_idx]['vel_x_smooth']) > vel_thresh * onset_offset_fraction:
        end_idx += 1
    
    onset_time = df.iloc[start_idx]['Seconds']
    offset_time = df.iloc[end_idx]['Seconds']
    
    # Calculate amplitude (absolute change in X position)
    start_x = df.iloc[start_idx]['X_smooth']
    end_x = df.iloc[end_idx]['X_smooth']
    amplitude = abs(end_x - start_x)
    
    downward_saccades.append({
        'time': peak_time,
        'velocity': peak_velocity,
        'start_time': onset_time,
        'end_time': offset_time,
        'duration': offset_time - onset_time,
        'start_position': start_x,
        'end_position': end_x,
        'amplitude': amplitude
    })

# Convert to DataFrames for easier handling
upward_saccades_df = pd.DataFrame(upward_saccades)
downward_saccades_df = pd.DataFrame(downward_saccades)

print(f"\n✅ Detected {len(pos_peaks)} upward saccades")
print(f"✅ Detected {len(neg_peaks)} downward saccades")

# Print summary statistics
if len(upward_saccades) > 0:
    print(f"\nUpward saccades - mean velocity: {upward_saccades_df['velocity'].mean():.2f} px/s")
    print(f"Upward saccades - mean duration: {upward_saccades_df['duration'].mean():.3f} s")
    print(f"Upward saccades - mean amplitude: {upward_saccades_df['amplitude'].mean():.2f} px")
    print(f"Upward saccades - std amplitude: {upward_saccades_df['amplitude'].std():.2f} px")

if len(downward_saccades) > 0:
    print(f"\nDownward saccades - mean velocity: {downward_saccades_df['velocity'].mean():.2f} px/s")
    print(f"Downward saccades - mean duration: {downward_saccades_df['duration'].mean():.3f} s")
    print(f"Downward saccades - mean amplitude: {downward_saccades_df['amplitude'].mean():.2f} px")
    print(f"Downward saccades - std amplitude: {downward_saccades_df['amplitude'].std():.2f} px")


Adaptive velocity threshold: 129.05 px/s
  (mean: 9.22 px/s, std: 29.96 px/s, k=4)
Saccade duration parameter: 0.2 (saccade ends when velocity < 25.81 px/s)

✅ Detected 113 upward saccades
✅ Detected 324 downward saccades

Upward saccades - mean velocity: 454.33 px/s
Upward saccades - mean duration: 0.052 s
Upward saccades - mean amplitude: 10.38 px
Upward saccades - std amplitude: 6.58 px

Downward saccades - mean velocity: -288.02 px/s
Downward saccades - mean duration: 0.074 s
Downward saccades - mean amplitude: 9.11 px
Downward saccades - std amplitude: 4.98 px


In [22]:
# SACCADE AMPLITUDE QC VISUALIZATION
#-------------------------------------------------------------------------------
# 1. Distribution of saccade amplitudes
# 2. Correlation between saccade amplitude and duration

from plotly.subplots import make_subplots

fig_qc = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Amplitude Distribution - Upward Saccades', 
        'Amplitude Distribution - Downward Saccades',
        'Amplitude vs Duration - Upward Saccades',
        'Amplitude vs Duration - Downward Saccades'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Amplitude distributions
if len(upward_saccades_df) > 0:
    # Histogram for upward saccades
    fig_qc.add_trace(
        go.Histogram(
            x=upward_saccades_df['amplitude'],
            nbinsx=30,
            name='Upward',
            marker_color='green',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Scatter plot for upward saccades
    fig_qc.add_trace(
        go.Scatter(
            x=upward_saccades_df['duration'],
            y=upward_saccades_df['amplitude'],
            mode='markers',
            name='Upward',
            marker=dict(color='green', size=8, opacity=0.6),
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Add correlation line for upward saccades
    corr_up = upward_saccades_df[['amplitude', 'duration']].corr().iloc[0, 1]
    z_up = np.polyfit(upward_saccades_df['duration'], upward_saccades_df['amplitude'], 1)
    p_up = np.poly1d(z_up)
    fig_qc.add_trace(
        go.Scatter(
            x=upward_saccades_df['duration'],
            y=p_up(upward_saccades_df['duration']),
            mode='lines',
            name=f'R={corr_up:.2f}',
            line=dict(color='darkgreen', width=2),
            showlegend=False
        ),
        row=2, col=1
    )

if len(downward_saccades_df) > 0:
    # Histogram for downward saccades
    fig_qc.add_trace(
        go.Histogram(
            x=downward_saccades_df['amplitude'],
            nbinsx=30,
            name='Downward',
            marker_color='purple',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Scatter plot for downward saccades
    fig_qc.add_trace(
        go.Scatter(
            x=downward_saccades_df['duration'],
            y=downward_saccades_df['amplitude'],
            mode='markers',
            name='Downward',
            marker=dict(color='purple', size=8, opacity=0.6),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Add correlation line for downward saccades
    corr_down = downward_saccades_df[['amplitude', 'duration']].corr().iloc[0, 1]
    z_down = np.polyfit(downward_saccades_df['duration'], downward_saccades_df['amplitude'], 1)
    p_down = np.poly1d(z_down)
    fig_qc.add_trace(
        go.Scatter(
            x=downward_saccades_df['duration'],
            y=p_down(downward_saccades_df['duration']),
            mode='lines',
            name=f'R={corr_down:.2f}',
            line=dict(color='darkviolet', width=2),
            showlegend=False
        ),
        row=2, col=2
    )

# Update layout
fig_qc.update_layout(
    title_text='Saccade Amplitude QC: Distributions and Correlations',
    height=800,
    showlegend=False
)

# Update axes labels
fig_qc.update_xaxes(title_text="Amplitude (px)", row=1, col=1)
fig_qc.update_xaxes(title_text="Amplitude (px)", row=1, col=2)
fig_qc.update_xaxes(title_text="Duration (s)", row=2, col=1)
fig_qc.update_xaxes(title_text="Duration (s)", row=2, col=2)
fig_qc.update_yaxes(title_text="Count", row=1, col=1)
fig_qc.update_yaxes(title_text="Count", row=1, col=2)
fig_qc.update_yaxes(title_text="Amplitude (px)", row=2, col=1)
fig_qc.update_yaxes(title_text="Amplitude (px)", row=2, col=2)

fig_qc.show()

# Print correlation statistics
print("\n=== SACCADE AMPLITUDE-DURATION CORRELATION ===\n")
if len(upward_saccades) > 0:
    print(f"Upward saccades (n={len(upward_saccades)}):")
    print(f"  Correlation (amplitude vs duration): {corr_up:.3f}")
    print(f"  Mean amplitude: {upward_saccades_df['amplitude'].mean():.2f} px")
    print(f"  Mean duration: {upward_saccades_df['duration'].mean():.3f} s")
    print(f"  Amp range: {upward_saccades_df['amplitude'].min():.2f} - {upward_saccades_df['amplitude'].max():.2f} px")

if len(downward_saccades) > 0:
    print(f"\nDownward saccades (n={len(downward_saccades)}):")
    print(f"  Correlation (amplitude vs duration): {corr_down:.3f}")
    print(f"  Mean amplitude: {downward_saccades_df['amplitude'].mean():.2f} px")
    print(f"  Mean duration: {downward_saccades_df['duration'].mean():.3f} s")
    print(f"  Amp range: {downward_saccades_df['amplitude'].min():.2f} - {downward_saccades_df['amplitude'].max():.2f} px")




=== SACCADE AMPLITUDE-DURATION CORRELATION ===

Upward saccades (n=113):
  Correlation (amplitude vs duration): 0.416
  Mean amplitude: 10.38 px
  Mean duration: 0.052 s
  Amp range: 2.21 - 29.77 px

Downward saccades (n=324):
  Correlation (amplitude vs duration): 0.593
  Mean amplitude: 9.11 px
  Mean duration: 0.074 s
  Amp range: 2.19 - 26.50 px


In [ ]:
# ADD SACCADE INDICATORS TO VideoData1
#-------------------------------------------------------------------------------
# Add boolean columns and speed columns to VideoData1 at their respective indices

# Initialize columns with False and NaN
VideoData1['saccade_up'] = False
VideoData1['saccade_down'] = False
VideoData1['saccade_speed_up'] = np.nan
VideoData1['saccade_speed_down'] = np.nan

# Mark upward saccades
for idx, row in upward_saccades_df.iterrows():
    # Find all indices within the saccade duration
    start_time = row['start_time']
    end_time = row['end_time']
    peak_time = row['time']
    peak_velocity = row['velocity']
    
    # Find the peak index
    peak_idx = VideoData1[VideoData1['Seconds'] == peak_time].index
    if len(peak_idx) > 0:
        peak_idx = peak_idx[0]
        # Mark the peak with speed
        VideoData1.loc[peak_idx, 'saccade_speed_up'] = peak_velocity
    
    # Mark all points within the saccade duration
    mask = (VideoData1['Seconds'] >= start_time) & (VideoData1['Seconds'] <= end_time)
    VideoData1.loc[mask, 'saccade_up'] = True

# Mark downward saccades
for idx, row in downward_saccades_df.iterrows():
    start_time = row['start_time']
    end_time = row['end_time']
    peak_time = row['time']
    peak_velocity = row['velocity']
    
    # Find the peak index
    peak_idx = VideoData1[VideoData1['Seconds'] == peak_time].index
    if len(peak_idx) > 0:
        peak_idx = peak_idx[0]
        # Mark the peak with speed
        VideoData1.loc[peak_idx, 'saccade_speed_down'] = peak_velocity
    
    # Mark all points within the saccade duration
    mask = (VideoData1['Seconds'] >= start_time) & (VideoData1['Seconds'] <= end_time)
    VideoData1.loc[mask, 'saccade_down'] = True

print(f"✅ Added saccade indicators to VideoData1")
print(f"\nSummary:")
print(f"   Upward saccades: {upward_saccades_df.shape[0]} events → {VideoData1['saccade_up'].sum()} timepoints marked")
print(f"   Downward saccades: {downward_saccades_df.shape[0]} events → {VideoData1['saccade_down'].sum()} timepoints marked")
print(f"   (Each saccade event spans multiple consecutive timepoints)")
print(f"\nSpeed values:")
print(f"   Upward speeds recorded: {VideoData1['saccade_speed_up'].notna().sum()} (peak velocities only)")
print(f"   Downward speeds recorded: {VideoData1['saccade_speed_down'].notna().sum()} (peak velocities only)")

# If VideoData2 exists, add saccade indicators to it as well
# Note: This assumes the same detection logic would apply to VideoData2
# You may want to repeat the detection for VideoData2 if it has separate data
if 'VideoData2_Has_Sleap' in globals() and VideoData2_Has_Sleap:
    VideoData2['saccade_up'] = False
    VideoData2['saccade_down'] = False
    VideoData2['saccade_speed_up'] = np.nan
    VideoData2['saccade_speed_down'] = np.nan
    print(f"\n⚠️ Note: VideoData2 saccade columns initialized but empty.")
    print(f"   To add saccades to VideoData2, run the detection code on VideoData2 data separately.")


In [ ]:
# VISUALIZE DETECTED SACCADES (Adaptive Method)
#-------------------------------------------------------------------------------
# Create overlay plot showing detected saccades with duration lines and peak arrows

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('X Position (px)', 'Velocity (px/s) with Detected Saccades')
)

# Add smoothed X position to the first subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['X_smooth'],
        mode='lines',
        name='Smoothed X',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

# Add smoothed velocity to the second subplot
fig.add_trace(
    go.Scatter(
        x=df['Seconds'],
        y=df['vel_x_smooth'],
        mode='lines',
        name='Smoothed Velocity',
        line=dict(color='red', width=2)
    ),
    row=2, col=1
)

# Add adaptive threshold lines for reference
fig.add_hline(
    y=vel_thresh,
    line_dash="dash",
    line_color="green",
    opacity=0.5,
    annotation_text=f"Adaptive threshold (±{vel_thresh:.0f} px/s)",
    row=2, col=1
)

fig.add_hline(
    y=-vel_thresh,
    line_dash="dash",
    line_color="green",
    opacity=0.5,
    row=2, col=1
)

# Set offset for saccade indicator lines (above the trace for upward, below for downward)
vel_max = df['vel_x_smooth'].max()
vel_min = df['vel_x_smooth'].min()
vel_range = vel_max - vel_min
line_offset = vel_range * 0.15  # 15% of velocity range

# Plot upward saccades with duration lines and peak arrows
if len(upward_saccades_df) > 0:
    for idx, row in upward_saccades_df.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']
        peak_time = row['time']
        peak_velocity = row['velocity']
        
        # Draw horizontal line spanning the saccade duration
        # Line is positioned above the velocity trace
        y_line_pos = vel_max + line_offset
        
        fig.add_shape(
            type="line",
            x0=start_time, y0=y_line_pos,
            x1=end_time, y1=y_line_pos,
            line=dict(color='green', width=3),
            row=2, col=1
        )
        
        # Add arrow annotation at the peak position pointing to the actual peak velocity
        # Arrow points from the line to the peak velocity value on the velocity trace
        y_arrow_start = y_line_pos
        y_arrow_end = peak_velocity
        
        fig.add_annotation(
            x=peak_time,
            y=y_arrow_start,
            ax=0,
            ay=y_arrow_end - y_arrow_start,  # arrow points to peak velocity
            arrowhead=2,  # filled arrowhead
            arrowsize=2,
            arrowwidth=2,
            arrowcolor='green',
            row=2, col=1,
            showarrow=True
        )
    
    # Add legend entry for upward saccades
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            name='Upward Saccades (duration lines)',
            marker=dict(symbol='line-ns', size=15, color='green', line=dict(width=3))
        ),
        row=2, col=1
    )

# Plot downward saccades with duration lines and peak arrows
if len(downward_saccades_df) > 0:
    for idx, row in downward_saccades_df.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']
        peak_time = row['time']
        peak_velocity = row['velocity']
        
        # Draw horizontal line spanning the saccade duration
        # Line is positioned below the velocity trace
        y_line_pos = vel_min - line_offset
        
        fig.add_shape(
            type="line",
            x0=start_time, y0=y_line_pos,
            x1=end_time, y1=y_line_pos,
            line=dict(color='purple', width=3),
            row=2, col=1
        )
        
        # Add arrow annotation at the peak position pointing to the actual peak velocity
        # Arrow points from the line to the peak velocity value on the velocity trace
        y_arrow_start = y_line_pos
        y_arrow_end = peak_velocity
        
        fig.add_annotation(
            x=peak_time,
            y=y_arrow_start,
            ax=0,
            ay=y_arrow_end - y_arrow_start,  # arrow points to peak velocity
            arrowhead=2,  # filled arrowhead
            arrowsize=2,
            arrowwidth=2,
            arrowcolor='purple',
            row=2, col=1,
            showarrow=True
        )
    
    # Add legend entry for downward saccades
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            name='Downward Saccades (duration lines)',
            marker=dict(symbol='line-ns', size=15, color='purple', line=dict(width=3))
        ),
        row=2, col=1
    )

# Update layout
fig.update_layout(
    title='Detected Saccades: Duration Lines + Peak Arrows (QC Visualization)',
    height=600,
    showlegend=True,
    legend=dict(x=0.01, y=0.99)
)

# Update axes
fig.update_xaxes(title_text="Time (s)", row=2, col=1)
fig.update_yaxes(title_text="X Position (px)", row=1, col=1)
fig.update_yaxes(title_text="Velocity (px/s)", row=2, col=1)

fig.show()


In [ ]:
VideoData1.columns


# PRE_ANR code

In [ ]:
# detect saccades 
df = VideoData1.copy()

df.index = pd.to_datetime(df.index * (1 / framerate), unit='s')

# 1) Compute velocity (units/s). Filter/diff/filter. Because sample rate is 1000 Hz, diff is * 1000.
window_size = int(round(2 / framerate * 1000))
df["velocity"] = df["Ellipse.Center.X"].rolling(window=window_size, center=True, min_periods=1).mean()
df["velocity"] = df["velocity"].diff() * 1000
window_size = int(round(4 / framerate * 1000))
df["velocity"] = df["velocity"].rolling(window=window_size, center=True, min_periods=1).mean()

# 2) Define a velocity threshold for saccades (adjust as needed)
# implement adaptive filter in the future 

# 3) Create a boolean mask for samples exceeding the threshold
df["is_saccade"] = df["velocity"].abs() > threshold

# 4) Group consecutive saccade samples to form saccade events.
#    Label each contiguous "True" block with a unique ID.
df["saccade_id"] = (df["is_saccade"] & ~df["is_saccade"].shift(fill_value=False)).cumsum() * df["is_saccade"]

# 5) Extract saccade onset times and basic details for each saccade.
saccade_events = []
for sacc_id, group in df.groupby("saccade_id"):
    if sacc_id == 0:
        continue
    saccade_time = group.index[0]
    peak_time = group["velocity"].abs().idxmax()  # Save the time when the absolute velocity peaks
    peak_velocity = group["velocity"].abs().max()
    direction = "positive" if group["velocity"].mean() > 0 else "negative"
    
    saccade_events.append({
        "saccade_id": sacc_id,
        "saccade_time": saccade_time,
        "peak_time": peak_time,         # New column for peak time
        "peak_velocity": peak_velocity,
        "direction": direction
    })

# 6) Apply a refractory period of 50 ms: if 2 saccade events occur within 50 ms, keep only the first.
filtered_saccade_events = []
last_event_time = None  # Initialize as None

for event in saccade_events:
    if last_event_time is None or (event["saccade_time"] - last_event_time) >= refractory_period:
        filtered_saccade_events.append(event)
        last_event_time = event["saccade_time"]


# 7) For each filtered saccade event, calculate the baseline and relative peak.
frame_duration = 1 / framerate  # seconds per frame
for event in filtered_saccade_events:
    saccade_time = event["saccade_time"]
    # Baseline: average the data for 3 frames immediately BEFORE the saccade onset.
    baseline_start = saccade_time - pd.Timedelta(seconds=3 * frame_duration)
    baseline = df.loc[baseline_start:saccade_time, "Ellipse.Center.X"].mean()
    event["baseline"] = baseline

    # Relative peak: in the next 40 ms after the saccade onset, measure the peak change relative to the baseline.
    window_end = saccade_time + pd.Timedelta(milliseconds=500)
    saccade_window = df.loc[saccade_time:window_end, "Ellipse.Center.X"]
    if event["direction"] == "positive":
        relative_peak = saccade_window.max() - baseline
    else:
        relative_peak = baseline - saccade_window.min()
    event["relative_peak"] = relative_peak

# Create a DataFrame of the filtered saccade events including the new metrics.
results_df = pd.DataFrame(filtered_saccade_events)


In [ ]:
if plot_saccade_detection_QC:
    pio.renderers.default = 'browser'
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df["velocity"], mode="lines", name="Velocity", line=dict(color="lightgrey", width=1)))
    fig.add_trace(go.Scatter(x=df.index, y=df["Ellipse.Center.X"], mode="lines", name="Ellipse.Center.X", line=dict(color="darkgrey", width=1), yaxis="y2"))
    if not results_df.empty:
        pos_df = results_df[results_df["direction"] == "positive"]
        neg_df = results_df[results_df["direction"] == "negative"]
        if not pos_df.empty:
            pos_starts = pos_df["saccade_time"]
            pos_y = df.loc[pos_starts, "Ellipse.Center.X"]
            fig.add_trace(go.Scatter(x=pos_starts, y=pos_y, mode="markers",
                                     marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="red")),
                                     name="Positive Saccade Onsets", yaxis="y2"))
        if not neg_df.empty:
            neg_starts = neg_df["saccade_time"]
            neg_y = df.loc[neg_starts, "Ellipse.Center.X"]
            fig.add_trace(go.Scatter(x=neg_starts, y=neg_y, mode="markers",
                                     marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="blue")),
                                     name="Negative Saccade Onsets", yaxis="y2"))
    fig.update_layout(
        title="Velocity and Ellipse.Center.X",
        xaxis_title="Time",
        yaxis=dict(title="Velocity"),
        yaxis2=dict(title="Ellipse.Center.X", overlaying="y", side="right")
    )
    fig.show()


In [ ]:
############################################################################################################
# INVESTIGATE issue of long stretches of consecutive very low inference predicition scores 
############################################################################################################


score_cutoff = 0.0000001
columns_of_interest = ['left.score','center.score','right.score','p1.score','p2.score','p3.score','p4.score','p5.score','p6.score','p7.score','p8.score']
total_points = len(VideoData1)


for col in columns_of_interest:
    count_below_threshold = (VideoData1[col] < score_cutoff).sum()
    percentage_below_threshold = (count_below_threshold / total_points) * 100
    
    # Find the longest consecutive series below threshold
    below_threshold = VideoData1[col] < score_cutoff
    longest_series = 0
    current_series = 0
    
    for value in below_threshold:
        if value:
            current_series += 1
            if current_series > longest_series:
                longest_series = current_series
        else:
            current_series = 0
    
    print(f"Column: {col} | Values below {score_cutoff}: {count_below_threshold} ({percentage_below_threshold:.2f}%) | Longest consecutive frame series: {longest_series}")


In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)